In [ ]:
using Agents, Random, Distributions, InteractiveDynamics;
using OpenStreetMapXPlot
using Plots
gr()

In [ ]:
@agent Transportor OSMAgent begin
    capacitate_m3::Float16
    capacitate_consumata_m3::Float16
    prob_livrare_corecta::Float16
    timpi_transfer_pachet_s::Vector{Float16}
end

In [ ]:
@agent Depozitar OSMAgent begin
    capacitate_m3::Float16
    capacitate_consumata_m3::Float16
    prob_livrare_corecta::Float16
    timpi_transfer_pachet_s::Vector{Float16}
end

In [ ]:
@agent Colet OSMAgent begin
    volum_m3::Float16
    valoare_ron::Float16
end

In [ ]:
Array{Tuple{Int,Int,Float64}}()

In [ ]:
function initialise(; map_path = "map.osm")
    model = ABM(Union{Transportor, Depozitar, Colet}, OpenStreetMapSpace(map_path))
    d_prob_livrare_corecta = truncated(Normal(.8, 2), .4, 1.)
    d_capacitate = truncated(Normal(20, 10), 5, 100)
    d_timpi_transfer = truncated(Exponential(60), 5, 60*15)
    puncte_pornire = Array{Tuple{Int,Int,Float64}}()
    for id in 1:10
        println(id)
        adresa = random_position(model)
        append!(puncte_pornire, adresa)
        ruta = OSM.plan_route(adresa, adresa, model)
        capacitate = rand(d_capacitate)
        prob_livrare = rand(d_prob_livrare_corecta)
        timp_transfer = rand(d_timpi_transfer, 1)
        depozitar = Depozitar(id, adresa, ruta, adresa, capacitate, 0., prob_livrare, timp_transfer)
        add_agent_pos!(depozitar, model)
    end
    # Colete
    d_volum = truncated(Normal(.1, .1), .01, 2)
    d_valoare = truncated(Normal(50, 50), 5, 10000)
    for id in 100:200
        inceput = sample(puncte_pornire)
        sfarsit = random_position(model)
        ruta = OSM.plan_route(inceput, sfarsit, model)
        volum = rand(d_volum)
        valoare = rand(d_valoare)
        colet = Colet(id, inceput, ruta, sfarsit, volum, valoare)
        add_agent_pos!(colet, model)
    end
    # Transportori
    d_capacitate = truncated(Normal(5, 2), 1, 20)
    for id in 11:30
        inceput = random_position(model)
        sfarsit = random_position(model)
        ruta = OSM.plan_route(inceput, sfarsit, model)
        capacitate = rand(d_capacitate)
        prob_livrare = rand(d_prob_livrare_corecta)
        timp_transfer = rand(d_timpi_transfer, 1)
        transportor = Transportor(id, inceput, ruta, sfarsit, capacitate, 0., prob_livrare, timp_transfer)
        add_agent_pos!(transportor, model)
    end
    return model
end

In [ ]:
m = initialise()

In [ ]:
a = m[1]

In [ ]:
a.

In [ ]:
function plotagents(model)
    ids = model.scheduler(model)
    markers = :circle
    pos = [OSM.map_coordinates(model[i], model) for i in ids]

    scatter!(
        pos;
        markershapes = markers,
        label = "",
        markerstrokewidth = 0.5,
        markerstrokecolor = :black,
        markeralpha = 0.7,
    )
end

In [ ]:
plotmap(m.space.m)
plotagents(m)